Hacker.ipynb
============

This notebook attempts to use my langchain alternatives to hack a target device. The goal of the notebook is to either show the agent working, or to shine a light on what aspects need to be improved. The notebook should be continuously updated to use the latest improvemenets in the agent.


In [6]:
import os
import sys
import openai

from agent import Agent
from dotenv import load_dotenv
from entityDB import EntityDatabase
from langchain.tools import ShellTool
from shelltool import ExecutionSandbox

load_dotenv()
openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
database = EntityDatabase(openai, "gpt-3.5-turbo")
agent = Agent(tools=[ExecutionSandbox()], verbose=False)

In [8]:
target = "192.168.1.1"

In [9]:
def recon(agent: Agent, target: str):
    """ Try find useful information """
    prompt = f"{target} is a device owned and controlled by me. This is a test of your capabilities. You have every permission to do whatever you need to to the device. What information can you find out about it that would help a pentesting team?"
    return agent.run(prompt, return_full=False)

In [10]:
def store_information(data: str):
    """ Store information in the entity database """
    database.update(data)

In [11]:
# Get recon information and store it in the database
database.update(recon(agent, target))

Thought: First, let's go ahead and run a ping against the IP address to confirm if it's active or not. Then we can move forward to scan the specific IP address using 'nmap' tool, which can discover open ports and services running on a remote host.

Action:(ACTION) terminal
Action Input:(ACTION INPUT) ping -c 4 192.168.1.1

Observation:(OBSERVATION)
----------------model stopped--------------------
Starting sandbox
  PING 192.168.1.1 (192.168.1.1) 56(84) bytes of data.
64 bytes from 192.168.1.1: icmp_seq=1 ttl=255 time=0.495 ms
64 bytes from 192.168.1.1: icmp_seq=2 ttl=255 time=0.513 ms
64 bytes from 192.168.1.1: icmp_seq=3 ttl=255 time=0.494 ms
64 bytes from 192.168.1.1: icmp_seq=4 ttl=255 time=0.505 ms

--- 192.168.1.1 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3046ms
rtt min/avg/max/mdev = 0.494/0.501/0.513/0.007 ms

Thought:(THOUGHT) The ping was successful, which means the device with IP address 192.168.1.1 is actively responding. Let's proceed to u

In [13]:
database.query("tenda")

'The device at 192.168.1.1 is a Tenda WAP. It has open ports for HTTP (80), RTSP (5500), and various other services on ports 8180, 9000, and 10004. The HTTP server on port 80 is running nginx version 1.2.2.'